# Step 8: Warp the moving image

In [1]:
import os
import zarr
from precomputed_tif.zarr_stack import ZarrStack
from phathom import io
from phathom.registration import registration
from phathom.utils import pickle_load

In [2]:
working_dir = '/home/jswaney/coregistration'

In [3]:
# Open images
fixed_zarr_path = 'fixed/zarr_stack/1_1_1'
moving_zarr_path = 'moving/zarr_stack/1_1_1'

fixed_img = io.zarr.open(os.path.join(working_dir, fixed_zarr_path), 
                         mode='r')
moving_img = io.zarr.open(os.path.join(working_dir, moving_zarr_path), 
                          mode='r')

In [4]:
# Load the coordinate interpolator
interpolator_path = 'interpolator.pkl'

interpolator = pickle_load(os.path.join(working_dir, 
                                        interpolator_path))

In [5]:
# Create a new zarr array for the registered image
nonrigid_zarr_path = 'moving/registered/1_1_1'

nonrigid_img = io.zarr.new_zarr(os.path.join(working_dir, 
                                             nonrigid_zarr_path),
                                fixed_img.shape,
                                fixed_img.chunks,
                                fixed_img.dtype)

In [ ]:
# Warp the entire moving image
nb_workers = 8
batch_size = None
padding = 4

registration.register(moving_img, 
                      nonrigid_img, 
                      interpolator, 
                      nb_workers, 
                      batch_size=batch_size, 
                      padding=padding)

In [ ]:
# Make the mipmap stack for neuroglancer
n_levels = 7
reg_dest_path = 'moving/registered'
reg_src_path = nonrigid_zarr_path  # written to 1_1_1 already

reg_src = zarr.NestedDirectoryStore(os.path.join(working_dir, 
                                                 reg_src_path))
reg_dest = os.path.join(working_dir, reg_dest_path)
reg_stack = ZarrStack(reg_src, reg_dest)

# Skip level 1
reg_stack.write_info_file(n_levels)
for level in range(2, n_levels+1):
    reg_stack.write_level_n(level)